In [1]:
import platform,os
os_name = platform.system()
hdfs_home = "/user/" + os.getenv("HOME").split("/")[2]

In [2]:
print(hdfs_home)
print(os_name)

/user/kranthidr
Linux


In [3]:
path = hdfs_home+"/dataSets/spark-guide/flight-data/parquet/2010-summary.parquet"

In [4]:
spark_home = "/usr/hdp/current/spark2-client"

In [5]:
import findspark
findspark.init(spark_home)
#findspark.init()
findspark.find()

'/usr/hdp/current/spark2-client'

In [6]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("sg14-rddSharedVariables").getOrCreate()
#spark = SparkSession.builder.master("local[*]").appName("sg14-rddSharedVariables").getOrCreate()

In [7]:
sc = spark.sparkContext

In [8]:
for x in sc._conf.getAll():
    if '/proxy/' in x[1]:
        print(x[1])

http://rm01.itversity.com:19288/proxy/application_1533622723243_20768


In [9]:
my_collection = "Spark The Definitive Guide : Big Data Processing Made Simple".split(" ")
words = spark.sparkContext.parallelize(my_collection, 2)

In [10]:
# COMMAND ----------

supplementalData = {"Spark":1000, "Definitive":200,
                    "Big":-300, "Simple":100}

In [11]:
# COMMAND ----------

suppBroadcast = spark.sparkContext.broadcast(supplementalData)

In [12]:
# COMMAND ----------

suppBroadcast.value

{'Big': -300, 'Definitive': 200, 'Simple': 100, 'Spark': 1000}

In [13]:
# COMMAND ----------

words.map(lambda word: (word, suppBroadcast.value.get(word, 0)))\
  .sortBy(lambda wordPair: wordPair[1])\
  .collect()

[('Big', -300),
 ('Data', 0),
 ('Processing', 0),
 ('Made', 0),
 ('The', 0),
 ('Guide', 0),
 (':', 0),
 ('Simple', 100),
 ('Definitive', 200),
 ('Spark', 1000)]

In [14]:
# COMMAND ----------

flights = spark.read.parquet(path)

In [15]:
# COMMAND ----------
accChina = spark.sparkContext.accumulator(0)

In [16]:
# COMMAND ----------

def accChinaFunc(flight_row):
  destination = flight_row["DEST_COUNTRY_NAME"]
  origin = flight_row["ORIGIN_COUNTRY_NAME"]
  if destination == "China":
    accChina.add(flight_row["count"])
  if origin == "China":
    accChina.add(flight_row["count"])

In [17]:
# COMMAND ----------

flights.foreach(lambda flight_row: accChinaFunc(flight_row))

In [18]:
# COMMAND ----------

accChina.value # 953


# COMMAND ----------

953